> Aprenentatge Computacional

**Nom**: Gabriel Rios Sanchez\
**NIU**: 1671177

# SUPERMARKET SALES

---

## 0. ENUNCIAT INICIAL

---

## 1. ANÀLISI DE DADES

---

## 2. PREPROCESSAT

---

## 3. SELECCIÓ DE MODELS

---

## 4. CONCLUSIÓ